In [2]:
#Created by Diogo Loureiro
#To use this script, please execute following steps:
    #1) Install requirements.txt
    #2) Save desired .csv files on data folder
    #3) Run this script
    #4) Reports will be generated inside reports folder (.html format)

#Documentation for both libs (pandas profilling and sweet viz
#Pandas profilling = https://ydata-profiling.ydata.ai/docs/master/
#Sweetviz = https://pypi.org/project/sweetviz/#description

import os
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import sweetviz as sv

#Generate sweetviz version? If not, change to False
sweetviz=False

# Get the current working directory
directory = os.getcwd()
print(directory)

# Create the 'reports' folder if it doesn't exist
reports_directory = os.path.join(directory, 'reports')
if not os.path.exists(reports_directory):
    os.makedirs(reports_directory)

# Set the 'data' directory
data_directory = os.path.join(directory, 'data')

# Iterate over all files in the 'data' directory
for filename in os.listdir(data_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_directory, filename)

        # Load the dataset
        data = pd.read_csv(file_path)

        # Generate the EDA report using pandas-profiling
        profile = ProfileReport(data,title=filename + ' Basic EDA',explorative=True,tsmode=True)

        #Check if sweetviz option is enabled
        if sweetviz==True:
            sv_profile = sv.analyze(data)

        # Save the report as an HTML file inside the 'reports' folder
        report_filename = os.path.splitext(filename)[0] + '_report.html'
        report_path = os.path.join(reports_directory, report_filename)
        sv_report_path = os.path.join(reports_directory, 'sv_'+report_filename)
        profile.to_file(report_path)

        #Check if sweetviz option is enabled
        if sweetviz==True:
            sv_profile.show_html(sv_report_path,open_browser=False)

        print("Report(s) saved at:", report_path)

c:\Users\Diogo\Desktop\Omdena\sao-paulo-chapter-passenger-demand\src\tasks\task_2_eda_dashboards


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\Diogo\.conda\envs\analytics\lib\site-packages\ydata_profiling\model\pandas\describe_timeseries_pandas.py:117: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  mad = pos_fft_above_med["ampl"].mad()
c:\Users\Diogo\.conda\envs\analytics\lib\site-packages\ydata_profiling\model\pandas\describe_timeseries_pandas.py:117: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  mad = pos_fft_above_med["ampl"].mad()


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Report(s) saved at: c:\Users\Diogo\Desktop\Omdena\sao-paulo-chapter-passenger-demand\src\tasks\task_2_eda_dashboards\reports\publiclines_pel_complete_report.html
